In [74]:
import pandas as pd
data = pd.read_csv("finaljoined.csv")

C:\Users\dofod\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [75]:
del data["Unnamed: 0"]

In [76]:
data.loanapply_insert_time = pd.to_datetime(data.loanapply_insert_time)
x = data.birth_year.astype(str)
x = x.str[:4]
data.birth_year = pd.to_datetime(x, format = "%Y")
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

In [77]:
import numpy as np
from catboost import CatBoost, Pool

In [78]:
income_dummy=pd.get_dummies(data['income_type'])

In [79]:
data=pd.concat([data,income_dummy],axis=1)

In [80]:
del data['income_type']

In [81]:
employment_dummy=pd.get_dummies(data['employment_type'])

data=pd.concat([data,employment_dummy],axis=1)

del data['employment_type']

In [82]:
house_dummy=pd.get_dummies(data['houseown_type'])

data=pd.concat([data,house_dummy],axis=1)

del data['houseown_type']

In [83]:
purpose_dummy=pd.get_dummies(data['purpose'])
purpose_dummy.rename(columns={"기타":"기타목적"},inplace=True)

data=pd.concat([data,purpose_dummy],axis=1)

del data['purpose']

In [84]:
data=data[data['insert_time']<'2022-06-01']
data=data.sort_values(by='is_applied')
data=data.drop_duplicates(subset='application_id',keep='last')

In [85]:
del data['loanapply_insert_time']
del data['bank_id']
del data['product_id']
del data['loan_limit']
del data['loan_rate']

In [86]:
data['is_applied'].mean()

0.5179389239664024

In [87]:
train_data=data[data['insert_time']<'2022-05-16']

In [88]:
train_data2=train_data.copy()

In [89]:
del train_data['is_applied']

In [90]:
test_data=data[data['insert_time']>='2022-05-16']
test_data=test_data[test_data['insert_time']<'2022-06-01']

In [91]:
test_data2=test_data.copy()

In [92]:
del test_data['is_applied']

In [93]:
train_label=train_data2['is_applied']

In [94]:
test_label=test_data2['is_applied']

In [95]:
from catboost import CatBoostClassifier

In [96]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [97]:
model.fit(train_data,train_label)

0:	learn: 0.6653837	total: 87.1ms	remaining: 87.1ms
1:	learn: 0.6608896	total: 160ms	remaining: 0us


In [98]:
preds_proba=model.predict_proba(test_data)

In [99]:
preds_proba

array([[0.55797681, 0.44202319],
       [0.55797681, 0.44202319],
       [0.39899535, 0.60100465],
       ...,
       [0.80182478, 0.19817522],
       [0.35458262, 0.64541738],
       [0.35458262, 0.64541738]])

In [100]:
preds_proba=preds_proba[:,1]

In [101]:
preds_proba

array([0.44202319, 0.44202319, 0.60100465, ..., 0.19817522, 0.64541738,
       0.64541738])

In [102]:
from sklearn.metrics import f1_score

In [103]:
f1_score(test_label,preds_proba>0.5179389239664024)

0.6468902814263878

In [104]:
preds_class=model.predict(test_data)

In [105]:
preds_class

array([0., 0., 1., ..., 0., 1., 1.])

In [106]:
f1_score(test_label,preds_class)

0.6530162579366632